# Import Packages and Start Exploring the Dataset

In [ ]:
'''
Import
'''
import numpy as np
import sklearn as sl
import pandas as pd 
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from google.colab import files
#from math import sqrt
#from sklearn import model_selection
#from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler


In [ ]:
'''
Mount drive
'''
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
'''
Read data
'''
train = pd.read_csv('/content/gdrive/MyDrive/Project 4/train.csv')
test = pd.read_csv('/content/gdrive/MyDrive/Project 4/test.csv')
sample = pd.read_csv('/content/gdrive/MyDrive/Project 4/sample.csv')

train

,id,country,description,points,price,province,variety
0,0,US,A nice starter Pinot for the price. Will intro...,86,14.0,California,Pinot Noir
1,1,Germany,This wine comes out of the bottle a little cra...,90,18.0,Mosel-Saar-Ruwer,Riesling
2,2,US,This easy-drinking Pinot Noir offers layers of...,85,28.0,California,Pinot Noir
3,3,France,"An attractive and ripely fruited Cab, this ope...",88,17.0,France Other,Cabernet Sauvignon
4,4,US,The vineyard is located in the western part of...,93,75.0,California,Pinot Noir
...,...,...,...,...,...,...,...
57051,57051,US,"Enticing, subtle pear, mango and nutmeg aromas...",92,40.0,California,Chardonnay
57052,57052,US,Very fresh on the nose but with an underlying ...,93,55.0,California,Chardonnay
57053,57053,US,Very delicious in terms of sheer flavor. Flood...,87,45.0,California,Pinot Noir
57054,57054,US,The nose of this bottling is quite shy. After ...,85,20.0,California,Cabernet Sauvignon


In [ ]:
'''
See what the output variable is: variety 
'''
sample

,id,variety
0,57056,Syrah
1,57057,Bordeaux-style Red Blend
2,57058,Cabernet Sauvignon
3,57059,Cabernet Sauvignon
4,57060,Rosé
...,...,...
14261,71317,Bordeaux-style Red Blend
14262,71318,Sauvignon Blanc
14263,71319,Syrah
14264,71320,Chardonnay


# Preprocessing

In [ ]:
'''
We were warned that there are missing values, so let's see how many/where the 
missing values are. 
'''
train.isna().sum()

id                0
country          23
description       0
points            0
price          3247
province         23
variety           0
dtype: int64

In [ ]:
'''
The suggestion is that we impute the data.

Country has 23 NAs, price has 3247 NAs, and province has 23 NAs. 

Some questions:
Should we remove all the NAs? If so, which ones?
Should we try to PREDICT the NA values? 

Let's do nothing for now (i.e., xgboost imputes the data).
'''



"\nThe suggestion is that we impute the data.\n\nCountry has 23 NAs, price has 3247 NAs, and province has 23 NAs. \n\nSome questions:\nShould we remove all the NAs? If so, which ones?\nShould we try to PREDICT the NA values? \n\nLet's do nothing for now (i.e., xgboost imputes the data).\n"

In [ ]:
'''
Combine train and test sets. 
'''
train_Y = train["variety"]
train_X = train.drop("variety", axis = 1)

temp = train_X.append(test)

temp

,id,country,description,points,price,province
0,0,US,A nice starter Pinot for the price. Will intro...,86,14.0,California
1,1,Germany,This wine comes out of the bottle a little cra...,90,18.0,Mosel-Saar-Ruwer
2,2,US,This easy-drinking Pinot Noir offers layers of...,85,28.0,California
3,3,France,"An attractive and ripely fruited Cab, this ope...",88,17.0,France Other
4,4,US,The vineyard is located in the western part of...,93,75.0,California
...,...,...,...,...,...,...
14261,71317,US,"The aromas of this rosé of Cabernet Sauvignon,...",86,14.0,Virginia
14262,71318,France,This powerful effort is holding back on its ri...,93,NaN,Bordeaux
14263,71319,US,This wine consists of all five red Bordeaux va...,88,55.0,California
14264,71320,US,"Here's a likeable Chard, with peach, pear and ...",85,19.0,California


In [ ]:
'''
Use TF-IDF on description column. 
'''

from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *
nltk.download('stopwords')

vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'), min_df=0.01)
temp_feat = vectorizer.fit_transform(temp['description'])

colnames = vectorizer.get_feature_names_out()

temp_feat_df = pd.DataFrame(temp_feat.toarray(), columns = colnames)

temp_feat_df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,10,100,20,2015,2016,2017,2018,2019,2020,accented,accents,acid,acidic,acidity,acids,across,add,adds,aftertaste,age,aged,aging,alcohol,almost,along,alongside,already,also,although,ample,anise,another,appealing,appellation,apple,apples,approachable,apricot,aroma,aromas,...,tones,top,touch,touches,tropical,two,valley,vanilla,varietal,variety,velvety,verdot,vibrant,vines,vineyard,vineyards,vintage,violet,warm,way,weight,well,wet,whiff,white,wild,wine,winemaker,winery,wines,without,wood,would,year,years,yellow,yet,young,zest,zesty
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.38495,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.180667,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.254038,0.0,0.0,0.0,0.0,0.0,0.0,0.086624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.118723,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.129354,0.000000,0.0,0.0,0.000000,0.0,0.222002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.287462,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.206819,0.0,0.232495,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.091492,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.203104,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71317,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.252731,0.0,0.0,0.270279,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.122852,...,0.0,0.0,0.192736,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.255432,0.0
71318,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.247625,0.0,0.0,0.0,0.000000,0.0
71319,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.

In [ ]:
''' 

Let's ignore the description variable for now - drop it.

Judge the performance before doing any text processing.

'''

temp_dropped = temp.drop("description", axis = 1)
temp_dropped = temp_dropped.reset_index(drop = True)

temp_dropped


,id,country,points,price,province
0,0,US,86,14.0,California
1,1,Germany,90,18.0,Mosel-Saar-Ruwer
2,2,US,85,28.0,California
3,3,France,88,17.0,France Other
4,4,US,93,75.0,California
...,...,...,...,...,...
71317,71317,US,86,14.0,Virginia
71318,71318,France,93,NaN,Bordeaux
71319,71319,US,88,55.0,California
71320,71320,US,85,19.0,California


In [ ]:
'''
Country and province are categorical variables.

Create dummies. 
'''
temp_dum = pd.get_dummies(temp_dropped)

temp_dum

,id,points,price,country_Argentina,country_Armenia,country_Australia,country_Austria,country_Brazil,country_Bulgaria,country_Canada,country_Chile,country_Croatia,country_Cyprus,country_Czech Republic,country_England,country_France,country_Georgia,country_Germany,country_Greece,country_Hungary,country_India,country_Israel,country_Italy,country_Lebanon,country_Luxembourg,country_Macedonia,country_Mexico,country_Moldova,country_Morocco,country_New Zealand,country_Peru,country_Portugal,country_Romania,country_Serbia,country_Slovakia,country_Slovenia,country_South Africa,country_Spain,country_Switzerland,country_Turkey,...,province_Urla-Thrace,province_Uruguay,province_Vale Trentino,province_Vale dos Vinhedos,province_Valle de Guadalupe,province_Veneto,province_Victoria,province_Vienna,province_Viile Timisului,province_Villány,province_Vin de Pays de Velvendo,province_Vinho Espumante de Qualidade,province_Vinho Verde,province_Vipavska Dolina,province_Virginia,province_Vlootenburg,province_Vânju Mare,province_Wachau,province_Wagram,province_Wagram-Donauland,province_Waiheke Island,province_Waipara,province_Waipara Valley,province_Wairarapa,province_Wairau Valley,province_Waitaki Valley,province_Walker Bay,province_Washington,province_Washington-Oregon,province_Weinland Österreich,province_Weinviertel,province_Wellington,province_Western Australia,province_Western Cape,province_Württemberg,province_Zenata,province_Österreichischer Perlwein,province_Österreichischer Sekt,province_Štajerska,province_Župa
0,0,86,14.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,90,18.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,85,28.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,88,17.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,93,75.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71317,71317,86,14.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
71318,71318,93,NaN,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
71319,71319,88,55.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
71320,71320,85,19.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
'''
Check if there are any of the same column names in either dataset.
'''
for x in temp_feat_df.columns:
  if x in temp_dum.columns:
    print(x)

price


In [ ]:
'''
Change price column name.
'''
temp_dum = temp_dum.rename(columns = {"price":"actual_Price"})

temp_dum

,id,points,actual_Price,country_Argentina,country_Armenia,country_Australia,country_Austria,country_Brazil,country_Bulgaria,country_Canada,country_Chile,country_Croatia,country_Cyprus,country_Czech Republic,country_England,country_France,country_Georgia,country_Germany,country_Greece,country_Hungary,country_India,country_Israel,country_Italy,country_Lebanon,country_Luxembourg,country_Macedonia,country_Mexico,country_Moldova,country_Morocco,country_New Zealand,country_Peru,country_Portugal,country_Romania,country_Serbia,country_Slovakia,country_Slovenia,country_South Africa,country_Spain,country_Switzerland,country_Turkey,...,province_Urla-Thrace,province_Uruguay,province_Vale Trentino,province_Vale dos Vinhedos,province_Valle de Guadalupe,province_Veneto,province_Victoria,province_Vienna,province_Viile Timisului,province_Villány,province_Vin de Pays de Velvendo,province_Vinho Espumante de Qualidade,province_Vinho Verde,province_Vipavska Dolina,province_Virginia,province_Vlootenburg,province_Vânju Mare,province_Wachau,province_Wagram,province_Wagram-Donauland,province_Waiheke Island,province_Waipara,province_Waipara Valley,province_Wairarapa,province_Wairau Valley,province_Waitaki Valley,province_Walker Bay,province_Washington,province_Washington-Oregon,province_Weinland Österreich,province_Weinviertel,province_Wellington,province_Western Australia,province_Western Cape,province_Württemberg,province_Zenata,province_Österreichischer Perlwein,province_Österreichischer Sekt,province_Štajerska,province_Župa
0,0,86,14.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,90,18.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,85,28.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,88,17.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,93,75.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71317,71317,86,14.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
71318,71318,93,NaN,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
71319,71319,88,55.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
71320,71320,85,19.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
'''
Combine text df and train df.
'''
temp_concat = pd.concat([temp_feat_df, temp_dum], axis = 1)

temp_concat

,10,100,20,2015,2016,2017,2018,2019,2020,accented,accents,acid,acidic,acidity,acids,across,add,adds,aftertaste,age,aged,aging,alcohol,almost,along,alongside,already,also,although,ample,anise,another,appealing,appellation,apple,apples,approachable,apricot,aroma,aromas,...,province_Urla-Thrace,province_Uruguay,province_Vale Trentino,province_Vale dos Vinhedos,province_Valle de Guadalupe,province_Veneto,province_Victoria,province_Vienna,province_Viile Timisului,province_Villány,province_Vin de Pays de Velvendo,province_Vinho Espumante de Qualidade,province_Vinho Verde,province_Vipavska Dolina,province_Virginia,province_Vlootenburg,province_Vânju Mare,province_Wachau,province_Wagram,province_Wagram-Donauland,province_Waiheke Island,province_Waipara,province_Waipara Valley,province_Wairarapa,province_Wairau Valley,province_Waitaki Valley,province_Walker Bay,province_Washington,province_Washington-Oregon,province_Weinland Österreich,province_Weinviertel,province_Wellington,province_Western Australia,province_Western Cape,province_Württemberg,province_Zenata,province_Österreichischer Perlwein,province_Österreichischer Sekt,province_Štajerska,province_Župa
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.180667,0.000000,0.0,0.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.118723,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.129354,0.000000,0.0,0.0,0.000000,0.0,0.222002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.287462,0.0,0.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71317,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.252731,0.0,0.0,0.270279,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.122852,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
71318,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
71319,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
71320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0,0,

In [ ]:
'''
Use simple imputer. 
'''
from sklearn.impute import SimpleImputer

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(temp_concat)

colnames = temp_concat.columns
temp_imp = pd.DataFrame(imp.transform(temp_concat), columns = colnames)

temp_imp 

,10,100,20,2015,2016,2017,2018,2019,2020,accented,accents,acid,acidic,acidity,acids,across,add,adds,aftertaste,age,aged,aging,alcohol,almost,along,alongside,already,also,although,ample,anise,another,appealing,appellation,apple,apples,approachable,apricot,aroma,aromas,...,province_Urla-Thrace,province_Uruguay,province_Vale Trentino,province_Vale dos Vinhedos,province_Valle de Guadalupe,province_Veneto,province_Victoria,province_Vienna,province_Viile Timisului,province_Villány,province_Vin de Pays de Velvendo,province_Vinho Espumante de Qualidade,province_Vinho Verde,province_Vipavska Dolina,province_Virginia,province_Vlootenburg,province_Vânju Mare,province_Wachau,province_Wagram,province_Wagram-Donauland,province_Waiheke Island,province_Waipara,province_Waipara Valley,province_Wairarapa,province_Wairau Valley,province_Waitaki Valley,province_Walker Bay,province_Washington,province_Washington-Oregon,province_Weinland Österreich,province_Weinviertel,province_Wellington,province_Western Australia,province_Western Cape,province_Württemberg,province_Zenata,province_Österreichischer Perlwein,province_Österreichischer Sekt,province_Štajerska,province_Župa
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.180667,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.118723,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.129354,0.000000,0.0,0.0,0.000000,0.0,0.222002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.287462,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71317,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.252731,0.0,0.0,0.270279,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.122852,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71318,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [ ]:
'''
Standardize
'''
ids = test["id"]
temp_imp = temp_imp.drop("id", axis = 1)

# Import the scaler to begin standardization 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

col_names = temp_imp.columns

# Apply the scaler to the merged data set
min_max_scaler = preprocessing.MinMaxScaler()
temp_norm = min_max_scaler.fit_transform(temp_imp)
temp_norm = pd.DataFrame(temp_norm, columns=col_names)

temp_norm

,10,100,20,2015,2016,2017,2018,2019,2020,accented,accents,acid,acidic,acidity,acids,across,add,adds,aftertaste,age,aged,aging,alcohol,almost,along,alongside,already,also,although,ample,anise,another,appealing,appellation,apple,apples,approachable,apricot,aroma,aromas,...,province_Urla-Thrace,province_Uruguay,province_Vale Trentino,province_Vale dos Vinhedos,province_Valle de Guadalupe,province_Veneto,province_Victoria,province_Vienna,province_Viile Timisului,province_Villány,province_Vin de Pays de Velvendo,province_Vinho Espumante de Qualidade,province_Vinho Verde,province_Vipavska Dolina,province_Virginia,province_Vlootenburg,province_Vânju Mare,province_Wachau,province_Wagram,province_Wagram-Donauland,province_Waiheke Island,province_Waipara,province_Waipara Valley,province_Wairarapa,province_Wairau Valley,province_Waitaki Valley,province_Walker Bay,province_Washington,province_Washington-Oregon,province_Weinland Österreich,province_Weinviertel,province_Wellington,province_Western Australia,province_Western Cape,province_Württemberg,province_Zenata,province_Österreichischer Perlwein,province_Österreichischer Sekt,province_Štajerska,province_Župa
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.247147,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.248050,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.296866,0.00000,0.0,0.0,0.000000,0.0,0.39182,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.443075,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71317,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.32145,0.0,0.0,0.466334,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.256678,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71318,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [ ]:
'''
The output variable is categorical. 

We can't apply dummy variables, so we need to encode the categories.
'''
from sklearn.preprocessing import LabelEncoder

marry_encoder = LabelEncoder()

marry_encoder.fit(train_Y)

train_Y_encoded = marry_encoder.transform(train_Y)

train_Y_encoded

array([4, 6, 4, ..., 4, 1, 1])

In [ ]:
'''
Split temp_norm back into train and test.
'''
train_X = temp_norm[0:len(train)]
test_X = temp_norm[len(train):len(train)+len(test)]

In [ ]:
'''
Split train into train and validation.
'''
x_train, x_val, y_train, y_val = train_test_split(train_X, train_Y_encoded, train_size=0.8)


# Modelling

## XGBOOST

In [ ]:
'''
Find how many unique output variables there are.
'''
len(np.unique(train_Y_encoded))

10

In [ ]:
'''
Fit the xgboost model.
'''
import xgboost as xgb

xgb_model = xgb.XGBClassifier(objective="multi:softmax", random_state=42, num_class = 10)
xgb_model.fit(x_train, y_train)
pred = xgb_model.predict(x_val)
print(pred)

[1 4 2 ... 3 1 8]


In [ ]:
'''
Evaluate the xgboost model using accuracy score.
'''
from sklearn.metrics import accuracy_score
print(accuracy_score(y_val, pred))

0.7992464072905713


In [ ]:
'''
Run the model on test data. 
'''
test_predictions = xgb_model.predict(test_X)
print(test_predictions)

[4 2 3 ... 9 2 1]


## Random Forest

In [ ]:
'''
Fit random forest model.

Need to impute data - RF doesn't do it automatically.
'''
from sklearn.ensemble import RandomForestClassifier

rfc=RandomForestClassifier(n_estimators=120, )
rfc.fit(x_train, y_train)
pred_rfc=rfc.predict(x_val)
print(pred_rfc)

[5 0 4 ... 6 5 2]


In [ ]:
'''
Evaluate the random forest model using accuracy score.
'''
from sklearn.metrics import accuracy_score
print(accuracy_score(y_val, pred_rfc))

0.8403434980722047


In [ ]:
'''
Run the model on test data. 
'''
test_predictions = rfc.predict(test_X)
print(test_predictions)

[4 2 3 ... 1 2 1]


## Multinominal Logit


In [ ]:
'''
Fit logit model.
'''
from sklearn.linear_model import LogisticRegression

logit = LogisticRegression(multi_class='multinomial', solver='newton-cg').fit(x_train, y_train)

logit_pred = logit.predict(x_val)

print(logit_pred)

[1 4 9 ... 4 2 4]


In [ ]:
'''
Evaluate the logit model using accuracy score.
'''
from sklearn.metrics import accuracy_score

print(accuracy_score(y_val, logit_pred))


0.8234314756396776




```
# This is formatted as code
```

## SVM 

In [ ]:
'''
Fit SVM model.
'''

from sklearn import svm

# Try two different kernels. 
rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(x_train, y_train)
poly = svm.SVC(kernel='poly', degree=3, C=1).fit(x_train, y_train)

poly_pred = poly.predict(x_dev)
rbf_pred = rbf.predict(x_dev)

print("Polynomial Kernel:", poly_pred)
print("RBF Kernel:", rbf_pred)

In [ ]:
'''
Evaluate the SVM model using accuracy score.
'''
from sklearn.metrics import accuracy_score

print("Polynomial Kernel:",accuracy_score(y_val, poly_pred))
print("RBF Kernel:",accuracy_score(y_val, rbf_pred))

## Voting Classifier

In [ ]:
len(x_train.columns.unique())

811

In [ ]:
'''
Take the four (xgboost, rf, svm - poly, svm - rbf) models 
and use voting to determine final output.
'''
from sklearn.ensemble import VotingClassifier
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Our classifiers.
xgb_model = xgb.XGBClassifier(objective="multi:softmax", random_state=42, num_class = 10)
rfc=RandomForestClassifier(n_estimators=100, )
logit = LogisticRegression(multi_class='multinomial', solver='newton-cg')

voting_classifiers = VotingClassifier(estimators=[('xgb', xgb_model), ('rf', rfc), ('logit', logit)],
                         voting='hard')

voting = voting_classifiers.fit(x_train, y_train)

vote_pred = voting.predict(x_val)
print(vote_pred)

[2 5 5 ... 4 4 5]


In [ ]:
'''
Evaluate the voting classifier using accuracy score.
'''
from sklearn.metrics import accuracy_score

print(accuracy_score(y_val, vote_pred))


0.8305292674377848


In [ ]:
'''
Run the model on test data. 
'''
test_predictions = voting.predict(test_X)
print(test_predictions)

[4 2 3 ... 1 2 1]


##K Neighbors Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=267, p = 2)
neigh.fit(x_train, y_train)

knn_pred = neigh.predict(x_val)
print(knn_pred)

[1 4 2 ... 2 2 4]


In [ ]:
'''
Evaluate the KNN classifier using accuracy score.

n_neighbors = 3, p = 2: 0.68
n_neighbors=5, p = 2: 0.71
n_neighbors=5, p = 1: 0.67
n_neighbors=267, p = 2: 0.68
'''
from sklearn.metrics import accuracy_score

print(accuracy_score(y_val, knn_pred))


0.681826147914476


# Submission

In [ ]:
'''
Example of submission file. 
'''
sample

,id,variety
0,57056,Syrah
1,57057,Bordeaux-style Red Blend
2,57058,Cabernet Sauvignon
3,57059,Cabernet Sauvignon
4,57060,Rosé
...,...,...
14261,71317,Bordeaux-style Red Blend
14262,71318,Sauvignon Blanc
14263,71319,Syrah
14264,71320,Chardonnay


In [ ]:
'''
The output variable is encoded, so we need to de-encode it.
'''
predictions = marry_encoder.inverse_transform(test_predictions)

In [ ]:
'''
Make the dataframe in the format of the submission.
'''
submission = pd.DataFrame(data = ids, columns=["id"])
submission["variety"] = predictions
submission

,id,variety
0,57056,Pinot Noir
1,57057,Chardonnay
2,57058,Merlot
3,57059,Red Blend
4,57060,Sauvignon Blanc
...,...,...
14261,71317,Bordeaux-style Red Blend
14262,71318,Bordeaux-style Red Blend
14263,71319,Cabernet Sauvignon
14264,71320,Chardonnay


In [ ]:
'''
Export dataframe.
'''
submission.to_csv('submission.csv', index = False )
from google.colab import files
files.download("submission.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>